## Aprendizagem Automática - Projeto

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Models and selection methods
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
# Linear regression metrics
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.preprocessing import LabelBinarizer

In [5]:
# Funcoes auxiliares

def printRegStatistics(truth, preds):
    print("The RVE is: ", explained_variance_score(truth, preds))
    print("The rmse is: ", mean_squared_error(truth, preds, squared=False))
    corr, pval = pearsonr(truth, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(truth, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(truth, preds))
    

#Devolve uma lista com quantos elementos há seguidos entre valores de NaN. p.e: [1,nan,2,3,nan,nan,2,nan,2,3,4] -> [1,2,1,3]
def count_not_nan(serie):
    conta=0
    lista_seguida=[]
    for i in range(len(serie)):
        if serie[i]==1:
            conta+=1
        else:
            if conta!=0:
                lista_seguida.append(conta)
                conta=0
            else:
                conta=0
                
        if i==len(serie)-1 and conta!=0:
            lista_seguida.append(conta)
    return lista_seguida

#Função que lida com os nan nas df. Se o país não tiver nenhum dado -> drop. Se o país não tiver pelo menos lag+1 dados seguidos
#-> drop. O resto fica

def drop_nan(df,lag):
    pais_nan=[]
    for pais in df["Country Name"].index:
        nan_val=list(pd.DataFrame(df.iloc[pais][4:]).count(axis=1)) #ver se os valores são nan
        if sum(nan_val)==0: #se for tudo nan
            pais_nan.append(pais)
        elif max(count_not_nan(nan_val))<=lag: #Se não houver mais do que lag valores seguidos, drop
            pais_nan.append(pais)

    df=df.drop(pais_nan, axis=0)
    df = df.reset_index(drop=True)
    return df

#Função que escolha o dataframe até um certo ano apenas, que remove os países que contenham nan nesse intervalo de tempo

def indexing(dataf, ano_i):
    X_list=dataf.drop(columns=dataf.iloc[:,[1,2,3]]+dataf.loc[:,[str(i) for i in range(1960,ano_i)]])
    X_list=X_list.T
    X_list=X_list.rename(columns=X_list.iloc[0]).drop(X_list.index[0])
    return X_list

#Cria um DataFrame com os dados de um país e respetivos lags

def lags(df, lag, país):
    new_df=pd.DataFrame()
    new_df["dados"]=df[país]
    colunas = [str(i) + "lag" for i in range(1,lag+1)]

    for i,k in enumerate(colunas):
        new_df[k] = new_df["dados"].shift(+i+1)
    new_df=new_df.dropna()
    return new_df

#Função que cria o dataset necessário para as timeseries. As time series são todas unidas numa só coluna e os países respetivos
# a essa série temporal recebem um 1. Também concatena os lags no início da df.

def time_series(df, lag):
    binario = LabelBinarizer().fit(df.columns)
    df_final= pd.DataFrame()
    for pais in df.columns:
        pais_bin=pd.DataFrame(data=list(binario.transform([pais]))*len(df[pais]),
                             columns=binario.classes_,
                             index=df.index)
        #Agora juntamos os lags e os dados respetivos
        df_mid=pd.DataFrame()
        df_mid=pd.concat((df_mid,lags(df, lag, pais),pais_bin),axis=1).dropna()

        #Finalmente, concatenamos tudo numa só DataFrame
        df_final = pd.concat((df_final, df_mid), sort=True)
        df_fina=df_final.dropna()
    return df_final

Importação dos datasets

In [18]:
# Dados 1960-2016
fertility = pd.read_csv("fertility_rate.csv")
population= pd.read_csv("country_population.csv")
expectancy = pd.read_csv("life_expectancy.csv")

fertility=drop_nan(fertility,4)
population=drop_nan(population,4)
expectancy=drop_nan(expectancy,4)

# Dados 1960-2020
fertility_2020 = pd.read_csv("fertility_2020.csv")
population_2020 = pd.read_csv("population_2020.csv")
expectancy_2020 = pd.read_csv("life_2020.csv")

#Criar os data frames

X_fert=indexing(fertility, 1960)
X_pop=indexing(population, 1960)
X_exp=indexing(expectancy, 1960)

fert_2020 = indexing(fertility_2020, 1960)
pop_2020 = indexing(population_2020, 1960)
exp_2020 = indexing(expectancy_2020, 1960)



Escolher a sample de 10 países aleatórios

In [7]:
sample = list(fertility["Country Name"].sample(n=10, random_state=483))
print(sample)

['Lesotho', 'North America', 'New Zealand', 'Singapore', 'Europe & Central Asia', 'Qatar', 'Uzbekistan', 'Estonia', 'Angola', 'Japan']


Vamos continuar a transformar o nosso dataset, de forma a que fiquem todas as séries temporais numa só coluna, a referência respetiva de que país se refere a série temporal e os respetivos lags para cada valor. 

In [9]:
#Versão final dos datasets
X_pop_fin=time_series(X_pop,4)
X_fert_fin=time_series(X_fert,4)
X_exp_fin=time_series(X_exp,4)

In [9]:
#X_pop_fin.to_csv("dfpop.csv")
#X_fert_fin.to_csv("dffert.csv")
#X_exp_fin.to_csv("dfexp.csv")

Com isto feito, podemos passar ao treino dos modelos

In [10]:
# Função para criar um dataframe X para um dado dataset, pais e ano
def createX(dataset : pd.DataFrame, country : str, year : int, lag) -> pd.DataFrame:
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = previousYearCountryData.copy()
    newX.index = [str(year)]
    for i in range(lag,2,-1):
        newX.at[str(year), str(i)+"lag"] = newX.at[str(year),str(i-1)+"lag"]
    newX.at[str(year), "1lag"] = newX.at[str(year),"dados"]
    newX.drop(["dados"], axis=1, inplace=True)
    return newX

# Função para prever dados novos para um dado dataset ano e pais a partir do modelo treinado dado
def predictAndCalculateDelta(model, dataset : pd.DataFrame, country : str, year : int, lag) -> float:
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = createX(dataset, country, year, lag)
    x = model.predict(newX)
    x_previous = previousYearCountryData.at[str(year-1),"dados"]
    delta = x - x_previous
    return x[0], delta[0]

# Imprime os resultados de forma agradavel
def printResults(prediction, real, country : str, year : int):
    pred_x, pred_delta = prediction
    real_x, real_delta = real
    abs_error, rel_error = calculateErrors(real_x, pred_x)
    print("Country: %s" % country)
    print("Year: %d" % year)
    print("Predictions".ljust(30) + "Real")
    print(("Data: %.2f" % pred_x).ljust(30) + ("%.2f" % real_x))
    print(("Delta: %.2f" % pred_delta).ljust(30) + ("%.2f" % real_delta))
    print("Error:   Absolute = %.2f    Relative = %.5f\n" % (abs_error, rel_error))

# Vai buscar o valor para um dado ano e pais e calcula o delta
def determineRealResults(dataset : pd.DataFrame, country : str, year : int):
    real_data = dataset.at[str(year), country]
    previous_year_data = dataset.at[str(year-1), country]
    delta = real_data - previous_year_data
    return real_data, delta

# Erro relativo e absoluto
def calculateErrors(real, pred):
    abs_error = abs(pred - real)
    relative_error = abs_error / real
    return abs_error, relative_error

Vamos passar à validação do modelo. Vamos separar o nosso dataset numa certa percentagem para training e outra para testing. Para tal, temos de voltar ao dataset e dividi-lo devidamente, fazendo uma divisão de 70% para training e 30% para testing.

In [13]:
#Cria os dataframes para o treino e teste

def set_train_test(df,lag): 
    train_set=pd.DataFrame()
    test_set=pd.DataFrame()
    paises=list(df.columns[lag:-1])
    
    for pais in paises:
        dados_pais=df[df[str(pais)]==1]
        pais_train= dados_pais[:int(0.7*(len(dados_pais.index)))] #Training set com 70% dos dados
        pais_test=dados_pais[int(0.7*(len(dados_pais.index))):] #Test de 30%
        
        train_set=pd.concat((train_set,pais_train))
        test_set=pd.concat((test_set, pais_test))
   
    return train_set, test_set
    

def valid_pais(df, df_train, df_test, país, lag):
    
    X_train, y_train = df_train[list(df_train.columns)[:-1]], df_train["dados"]
    X_test, y_test = df_test[list(df_test.columns)[:-1]], df_test["dados"]

    reg = LinearRegression()
    reg.fit(X_train, y_train)
    
    previsoes=[]
    anos=[int(i) for i in list(df_test[df_test[país]==1].index)] #lista de anos que vaõ ser testados
    for ano in anos:
        pred = predictAndCalculateDelta(reg, df, país, ano, lag)     
        previsoes.append(pred[0])
        
    y_test_país=df_test[df_test[país]==1]["dados"]
    return anos, previsoes, y_test_país

In [31]:
X_pop_train, X_pop_test= set_train_test(X_pop_fin,4)
y_pop_data = X_pop_train["dados"]
X_pop_data = X_pop_train[list(X_pop_train.columns)[:-1]]
reg_pop = LinearRegression()
reg_pop.fit(X_pop_data, y_pop_data)
random_country_pop_stats = []

for country in sample:
    anos, prevs, reais = valid_pais(X_pop_fin, X_pop_train, X_pop_test, country, 4)
    print("População")
    print(country)
    print("Estatisticas Modelo - Validação")
    printRegStatistics(reais, prevs)
    print("\nPrevisão 2017")
    pred = predictAndCalculateDelta(reg_pop, X_pop_fin, country, 2017, 4) #not sure se estou a usar o X_pop correto
    real = determineRealResults(pop_2020, country, 2017)
    random_country_pop_stats.append((country, pred, real))
    printResults(pred, real, country, 2017)


População
Lesotho
Estatisticas Modelo - Validação
The RVE is:  0.9976621491963789
The rmse is:  21631.08833345681
The Correlation Score is is: 0.9995 (p-value=2.577891e-22)

The Maximum Error is is:  31192.699299333617
The Mean Absolute Error is:  21120.732613201777

Previsão 2017
Country: Lesotho
Year: 2017
Predictions                   Real
Data: 2264969.03              2170617.00
Delta: 61148.03               26745.00
Error:   Absolute = 94352.03    Relative = 0.04347

População
North America
Estatisticas Modelo - Validação
The RVE is:  0.9992858314585599
The rmse is:  3395919.1858744347
The Correlation Score is is: 0.9998 (p-value=9.167084e-25)

The Maximum Error is is:  4113538.0594411492
The Mean Absolute Error is:  3376750.610767018

Previsão 2017
Country: North America
Year: 2017
Predictions                   Real
Data: 365023801.00            361731237.00
Delta: 5566308.00             2485441.00
Error:   Absolute = 3292564.00    Relative = 0.00910

População
New Zealand
Estati

In [32]:
X_fert_train, X_fert_test= set_train_test(X_fert_fin,4)
y_fert_data = X_fert_train["dados"]
X_fert_data = X_fert_train[list(X_fert_train.columns)[:-1]]
reg_fert = LinearRegression()
reg_fert.fit(X_fert_data, y_fert_data)
random_country_fert_stats = []

for country in sample:
    anos, prevs, reais = valid_pais(X_fert_fin, X_fert_train, X_fert_test, country, 4)
    print("Fertilidade")
    print(country)
    print("Estatisticas Modelo")
    printRegStatistics(reais, prevs)
    print("\nPrevisão 2017")
    pred = predictAndCalculateDelta(reg_fert, X_fert_fin, country, 2017, 4) #not sure se estou a usar o X_fert correto
    real = determineRealResults(fert_2020, country, 2017)
    random_country_fert_stats.append((country, pred, real))
    printResults(pred, real, country, 2017)

Fertilidade
Lesotho
Estatisticas Modelo
The RVE is:  0.996856075632792
The rmse is:  0.014290150783940456
The Correlation Score is is: 0.9985 (p-value=4.494444e-19)

The Maximum Error is is:  0.021795337722185337
The Mean Absolute Error is:  0.012643478338608516

Previsão 2017
Country: Lesotho
Year: 2017
Predictions                   Real
Data: 3.06                    3.19
Delta: -0.04                  -0.05
Error:   Absolute = 0.13    Relative = 0.04040

Fertilidade
North America
Estatisticas Modelo
The RVE is:  0.8614804825028723
The rmse is:  0.033988215157771756
The Correlation Score is is: 0.9442 (p-value=3.905419e-08)

The Maximum Error is is:  0.06961053898763891
The Mean Absolute Error is:  0.0273429922635689

Previsão 2017
Country: North America
Year: 2017
Predictions                   Real
Data: 1.76                    1.74
Delta: -0.02                  -0.05
Error:   Absolute = 0.01    Relative = 0.00772

Fertilidade
New Zealand
Estatisticas Modelo
The RVE is:  0.29121913988

In [33]:
X_exp_train, X_exp_test= set_train_test(X_exp_fin,4)
y_exp_data = X_exp_train["dados"]
X_exp_data = X_exp_train[list(X_exp_train.columns)[:-1]]
reg_exp = LinearRegression()
reg_exp.fit(X_exp_data, y_exp_data)
random_country_exp_stats = []

for country in sample:
    anos, prevs, reais = valid_pais(X_exp_fin, X_exp_train, X_exp_test, country, 4)
    print("Fertilidade")
    print(country)
    print("Estatisticas Modelo")
    printRegStatistics(reais, prevs)
    print("\nPrevisão 2017")
    pred = predictAndCalculateDelta(reg_exp, X_exp_fin, country, 2017, 4) #not sure se estou a usar o X_exp correto
    real = determineRealResults(exp_2020, country, 2017)
    random_country_exp_stats.append((country, pred, real))
    printResults(pred, real, country, 2017)

Fertilidade
Lesotho
Estatisticas Modelo
The RVE is:  0.9860794493829673
The rmse is:  0.3918438610819988
The Correlation Score is is: 0.9989 (p-value=3.858171e-20)

The Maximum Error is is:  0.6865034321603858
The Mean Absolute Error is:  0.30236747044739687

Previsão 2017
Country: Lesotho
Year: 2017
Predictions                   Real
Data: 54.60                   53.06
Delta: 0.42                   0.80
Error:   Absolute = 1.53    Relative = 0.02889

Fertilidade
North America
Estatisticas Modelo
The RVE is:  0.9551327981055259
The rmse is:  0.1509787809966557
The Correlation Score is is: 0.9800 (p-value=3.248806e-11)

The Maximum Error is is:  0.31178831416096386
The Mean Absolute Error is:  0.12588538947816463

Previsão 2017
Country: North America
Year: 2017
Predictions                   Real
Data: 78.99                   78.88
Delta: -0.06                  0.00
Error:   Absolute = 0.12    Relative = 0.00147

Fertilidade
New Zealand
Estatisticas Modelo
The RVE is:  0.9660750969314338